In [13]:
!pip install keras
!pip install numpy
!pip install matplotlib
!pip install glob
!pip install tensorflow
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [3]:
IMAGE_SIZE = [224, 224]

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Give dataset path Define the paths
train_path = '/content/drive/My Drive/datavgg/train'
test_path = '/content/drive/My Drive/datavgg/test'

In [10]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [14]:
import os

# Get the number of classes
folders = os.listdir('/content/drive/My Drive/datavgg/train')
print(len(folders))

5


In [15]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
from keras import optimizers


sgd = optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


In [18]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [19]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [26]:
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/datavgg/train',
target_size = (224, 224),
batch_size = 32,
class_mode = 'categorical')

Found 255 images belonging to 5 classes.


In [27]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/datavgg/test',
target_size = (224, 224),
batch_size = 32,
class_mode = 'categorical')

Found 190 images belonging to 5 classes.


In [34]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule(10,0.01))

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5',
                               verbose=1, save_best_only=True)
callbacks = [checkpoint]
start = datetime.now()
model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

<ipython-input-34-ab9fae811f8f>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
4/5 [=======================>......] - ETA: 38s - loss: 0.9215 - accuracy: 0.2471 


Epoch 1: val_loss improved from inf to 0.87278, saving model to mymodel.h5
5/5 [==============================] - 296s 60s/step - loss: 0.9215 - accuracy: 0.2471 - val_loss: 0.8728 - val_accuracy: 0.3368
Training completed in time:  0:04:56.717770


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
from PIL import Image
import numpy as np
from keras.preprocessing import image

img = Image.open('/content/drive/My Drive/datavgg/test/dog/image_58.jpg')
img = img.resize((224, 224), Image.ANTIALIAS)
img = np.array(img)
X = image.img_to_array(img)
X = np.expand_dims(X, axis=0)
X = preprocess_input(X)
preds = model.predict(X)
print(preds)

<ipython-input-45-972590511ad9>:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((224, 224), Image.ANTIALIAS)


1/1 [==============================] - 1s 995ms/step
[[1.2847619e-35 1.6615327e-29 2.6930341e-24 1.0000000e+00 0.0000000e+00]]
